In [1]:
import torch
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

In [2]:
torch.manual_seed(42)

In [3]:
cancer = load_breast_cancer()

In [4]:
X_train, X_test, y_train, y_test = train_test_split(cancer.data, 
                                                    cancer.target,
                                                   test_size=0.2,
                                                   random_state=42)

In [5]:
X_train.shape

(455, 30)

In [6]:
X_test.shape

(114, 30)

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
device

device(type='cuda')

In [9]:
X_train = torch.tensor(X_train, dtype=torch.float32, device=device)
X_test = torch.tensor(X_test, dtype=torch.float32, device=device)
y_train = torch.tensor(y_train, dtype=torch.float32, device=device)
y_test = torch.tensor(y_test, dtype=torch.float32, device=device)

In [10]:
X_train

tensor([[9.0290e+00, 1.7330e+01, 5.8790e+01,  ..., 1.7500e-01, 4.2280e-01,
         1.1750e-01],
        [2.1090e+01, 2.6570e+01, 1.4270e+02,  ..., 2.9030e-01, 4.0980e-01,
         1.2840e-01],
        [9.1730e+00, 1.3860e+01, 5.9200e+01,  ..., 5.0870e-02, 3.2820e-01,
         8.4900e-02],
        ...,
        [1.4290e+01, 1.6820e+01, 9.0300e+01,  ..., 3.3330e-02, 2.4580e-01,
         6.1200e-02],
        [1.3980e+01, 1.9620e+01, 9.1120e+01,  ..., 1.8270e-01, 3.1790e-01,
         1.0550e-01],
        [1.2180e+01, 2.0520e+01, 7.7220e+01,  ..., 7.4310e-02, 2.6940e-01,
         6.8780e-02]], device='cuda:0')

In [11]:
y_train.shape

torch.Size([455])

In [12]:
y_train = y_train.view(-1,1)
y_test = y_test.view(-1,1)

In [13]:
y_train.shape

torch.Size([455, 1])

In [14]:
input_features = X_train.shape[1]
outputs = y_train.shape[1]

In [15]:
input_features, outputs

(30, 1)

In [16]:
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd

In [17]:
class Model(nn.Module):
    
    def __init__(self, in_dims, out_dims):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(in_dims, 32)
        self.fc2 = nn.Linear(32, out_dims)
        
    def forward(self, x):
        out = self.fc1(x)
        out = torch.relu(out)
        out = self.fc2(out)
        return torch.sigmoid(out)

In [26]:
model = Model(input_features, outputs)

In [27]:
model.to(device)

Model(
  (fc1): Linear(in_features=30, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=1, bias=True)
)

In [28]:
learning_rate = 0.0001
iters = 1000
criterion = nn.BCELoss()

optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [29]:
for epoch in range(iters):
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)
    
    loss.backward()
    
    optimizer.step()
    optimizer.zero_grad()
    
    if epoch % 100 == 0:
        print("Epoch: {}, Loss: {}".format(epoch, loss))

Epoch: 0, Loss: 31.460100173950195
Epoch: 100, Loss: 1.1117026805877686
Epoch: 200, Loss: 0.8827389478683472
Epoch: 300, Loss: 0.6078259348869324
Epoch: 400, Loss: 0.33753105998039246
Epoch: 500, Loss: 0.71336430311203
Epoch: 600, Loss: 0.24982386827468872
Epoch: 700, Loss: 0.4410471022129059
Epoch: 800, Loss: 0.22094951570034027
Epoch: 900, Loss: 0.21264803409576416


In [30]:
from sklearn.metrics import accuracy_score

In [34]:
with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted = y_predicted.round()
    
    if torch.cuda.is_available():
        y_predicted = y_predicted.detach().cpu().numpy()
        
        
    print(f'Accuracy: {accuracy_score(y_test, y_predicted)}')

Accuracy: 0.9473684210526315
